# User Categories

Categoriasation of users according to the categories of the points of interest they visit.

In [1]:
import pandas as pd

### Data

In [2]:
walkins = pd.read_csv('Aueb Dmst Assignment Data\walkins.csv', dtype = str)
users = pd.read_csv('Aueb Dmst Assignment Data\\users.csv', dtype = str)
pois = pd.read_csv('Aueb Dmst Assignment Data\pois.csv', dtype = str)
categories = pd.read_csv('Aueb Dmst Assignment Data\categories.csv', dtype = str)
points = pd.read_csv('Aueb Dmst Assignment Data\\users.csv', dtype = str)

In [3]:
categories.rename(columns = {'id': 'category_id', 'title': 'category'}, inplace = True)
pois.rename(columns = {'id': 'poi_id'}, inplace = True)
pois_categories = pd.merge(pois[['poi_id', 'category_id', 'title']],
                           categories,
                           how = 'left',
                           on = 'category_id')

### User Categorisation

In [4]:
visits = pd.merge(walkins[['created', 'poi_id', 'user_id']],
                  pois_categories[['poi_id', 'category_id', 'category']],
                  how = 'left',
                  on = 'poi_id')
visits = visits[['created', 'user_id', 'category']]

In [5]:
users = visits.groupby(by=['user_id', 'category']).count().reset_index()
users.rename(columns = {'created': 'visits'}, inplace = True)

#### Woman

Users belong to this category if they fulfill at least one of the following:
* Visits in Beauty Stores > 2
* Visits in Women's Stores > 2

In [6]:
users['Woman'] = ((users['category'] == 'Women\'s Store') & (users['visits'] > 2)) | ((users['category'] == 'Beauty Store') & (users['visits'] > 2))


#### Parent

Users belong to this category if they fulfill at least one of the following:
* Visits in Playgrounds > 1
* Visits in Kids Stores > 2

In [7]:
users['Parent'] = ((users['category'] == 'Playground') & (users['visits'] > 3)) | ((users['category'] == 'Kids Store') & (users['visits'] > 2))

#### Pet Owner

Users belong to this category if they fulfill at least one of the following:
* Visits in Pet Stores > 2

In [8]:
users['Pet Owner'] = ((users['category'] == 'Pet Store') & (users['visits'] > 2))

#### Gamer

Users belong to this category if they fulfill at least one of the following:
* Visits in Gaming Cafes > 1

In [9]:
users['Gamer'] = ((users['category'] == 'Gaming Cafe') & (users['visits'] > 1))

#### Student

Users belong to this category if they fulfill at least one of the following:
* Visits in University > 1

In [10]:
users['Student'] = ((users['category'] == 'University') & (users['visits'] > 1))

#### Athletic

Users belong to this category if they fulfill at least one of the following:
* Visits in Gyms > 3

In [11]:
users['Athletic'] = ((users['category'] == 'Gym') & (users['visits'] > 3))

#### Foodie

Categoies of Points of Interest that fall into this category: Souvlaki, Taverna, Steakhouse, Italian, Restaurant, Ethnic, Burgers, Sandwich Place

Users belong to this category if they fulfill at least one of the following:
* Variety of categories visited > 3

In [12]:
users_foodie = users[(users['category'] == 'Souvlaki') |
                     (users['category'] == 'Taverna') |
                     (users['category'] == 'Steakhouse') |
                     (users['category'] == 'Italian') |
                     (users['category'] == 'Restaurant') |
                     (users['category'] == 'Ethnic') |
                     (users['category'] == 'Burgers') |
                     (users['category'] == 'Sandwich Place')]
users_foodie = users_foodie.groupby(by=['user_id']).count().reset_index()

In [13]:
users_foodie['variety'] = users_foodie['category'] > 3

In [14]:
users = pd.merge(users,
                 users_foodie[['user_id', 'variety']],
                 on = 'user_id',
                 how = 'left')

In [15]:
users['Foodie'] = False
users['Foodie'] = (users['variety'] == True)

#### Shopper

* Categoies of Points of Interest that fall into this category: Women's Store, Mobile Phone Shop, Shoe Store, Clothing Store, Beauty Store, Game Store, Electronics Store, Kids Store, Shopping Mall, Home Store

Users belong to this category if they fulfill at least one of the following:
* Variety of categories visited > 4

In [16]:
users_shopper = users[(users['category'] == ' Women\'s Store') |
                      (users['category'] == 'Mobile Phone Shop') |
                      (users['category'] == 'Shoe Store') |
                      (users['category'] == 'Clothing Store') |
                      (users['category'] == 'Beauty Store') |
                      (users['category'] == 'Game Store') |
                      (users['category'] == 'Electronics Store') | 
                      (users['category'] == 'Kids Store') |
                      (users['category'] == 'Shopping Mall') |
                      (users['category'] == 'Home Store')]
users_shopper = users_shopper.groupby(by=['user_id']).count().reset_index()

In [17]:
users_shopper['variety_2'] = users_shopper['category'] > 4

In [18]:
users = pd.merge(users,
                 users_shopper[['user_id', 'variety_2']],
                 on = 'user_id',
                 how = 'left')

In [19]:
users['Shopper'] = False
users['Shopper'] = (users['variety_2'] == True)

#### Final DataFrame

In [20]:
no_category = users[(users['Woman'] != True) & (users['Parent'] != True) & (users['Pet Owner'] != True) &
                    (users['Gamer'] != True) & (users['Student'] != True) & (users['Athletic'] != True) & 
                    (users['Foodie'] != True) & (users['Shopper'] != True)]
users = users[(users['Woman'] == True) | (users['Parent'] == True) | (users['Pet Owner'] == True) |
              (users['Gamer'] == True) | (users['Student'] == True) | (users['Athletic'] == True) | 
              (users['Foodie'] == True) | (users['Shopper'] == True)]
user_categories = users[['user_id', 'Woman', 'Parent', 'Pet Owner', 'Gamer', 'Student', 'Athletic', 'Foodie', 'Shopper']]
user_categories.drop_duplicates(inplace = True)
user_categories.replace({False: 0, True: 1}, inplace=True)
user_categories = user_categories.groupby(by=['user_id']).sum()

<ipython-input-20-e54053979ae9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_categories.drop_duplicates(inplace = True)
c:\users\theod\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


We only want values 0 and 1, where 0 is false and 1 is true.

In [21]:
user_categories.Foodie.unique()

array([0, 4, 3, 5, 6, 1], dtype=int64)

In [22]:
user_categories.Shopper.unique()

array([2, 4, 6, 3, 5, 0, 1], dtype=int64)

In [23]:
user_categories['Foodie'] = user_categories['Foodie'].replace([4, 3, 5, 6], 1)
user_categories['Shopper'] = user_categories['Shopper'].replace([2, 4, 6, 3, 5], 1)

In [24]:
user_categories.head(10)

,Woman,Parent,Pet Owner,Gamer,Student,Athletic,Foodie,Shopper
user_id,,,,,,,,
10,0,0,0,0,1,0,0,1
100,1,0,0,0,1,1,0,1
101,1,0,0,0,1,1,0,1
103,1,1,1,0,1,1,0,1
104,1,0,0,0,1,0,0,1
105,1,0,1,0,1,0,0,1
106,1,0,1,0,1,0,1,1
107,1,0,1,0,1,1,0,1
108,1,0,0,0,1,1,0,1


### Amount of users in each User Category

In [25]:
user_categories.sum().reset_index()

,index,0
0,Woman,707
1,Parent,46
2,Pet Owner,360
3,Gamer,27
4,Student,693
5,Athletic,346
6,Foodie,41
7,Shopper,689


### Users that do not belong in any of the User Categories

In [26]:
no_category.head(5)

,user_id,category,visits,Woman,Parent,Pet Owner,Gamer,Student,Athletic,variety,Foodie,variety_2,Shopper
46,10037,Supermarket,1,False,False,False,False,False,False,NaN,False,NaN,False
47,10079,Home Store,1,False,False,False,False,False,False,NaN,False,False,False
75,10163,Cafe,4,False,False,False,False,False,False,NaN,False,NaN,False
76,10163,Supermarket,7,False,False,False,False,False,False,NaN,False,NaN,False
77,10185,Supermarket,1,False,False,False,False,False,False,NaN,False,NaN,False


In [27]:
no_category.groupby(by = 'category').count().sort_values(by = 'user_id', ascending = False).head(10)

,user_id,visits,Woman,Parent,Pet Owner,Gamer,Student,Athletic,variety,Foodie,variety_2,Shopper
category,,,,,,,,,,,,
Supermarket,171,171,171,171,171,171,171,171,43,171,68,171
Park,130,130,130,130,130,130,130,130,23,130,49,130
Square,115,115,115,115,115,115,115,115,35,115,66,115
Metro Station,112,112,112,112,112,112,112,112,34,112,64,112
Cultural Center,108,108,108,108,108,108,108,108,22,108,43,108
Beach,103,103,103,103,103,103,103,103,19,103,39,103
Marina,87,87,87,87,87,87,87,87,12,87,24,87
Cafe,71,71,71,71,71,71,71,71,38,71,51,71
Landmark,63,63,63,63,63,63,63,63,4,63,23,63


In [28]:
no_category.groupby(by = 'user_id').count().sort_values(by = 'category', ascending = False).value_counts('category')

category
1     108
7      46
2      34
4      25
5      18
6      15
3      12
13      9
10      9
8       5
15      5
22      5
9       5
18      4
11      2
12      2
14      2
17      2
21      2
16      1
20      1
23      1
dtype: int64

**We reach the conclusion that there are no common visits by these users and that is the reason we cannot include them in any of the categories or create any new ones.**

### Overlapping categories

Checking the number of users of each User Category that, at the same time, belong to other User Categories.

#### Woman

In [29]:
user_categories[user_categories['Woman'] == 1].sum().reset_index()

,index,0
0,Woman,707
1,Parent,39
2,Pet Owner,341
3,Gamer,24
4,Student,684
5,Athletic,338
6,Foodie,40
7,Shopper,679


#### Parent

In [30]:
user_categories[user_categories['Parent'] == 1].sum().reset_index()

,index,0
0,Woman,39
1,Parent,46
2,Pet Owner,23
3,Gamer,3
4,Student,29
5,Athletic,19
6,Foodie,2
7,Shopper,35


#### Pet Owner

In [31]:
user_categories[user_categories['Pet Owner'] == 1].sum().reset_index()

,index,0
0,Woman,341
1,Parent,23
2,Pet Owner,360
3,Gamer,16
4,Student,332
5,Athletic,171
6,Foodie,20
7,Shopper,332


#### Gamer

In [32]:
user_categories[user_categories['Gamer'] == 1].sum().reset_index()

,index,0
0,Woman,24
1,Parent,3
2,Pet Owner,16
3,Gamer,27
4,Student,18
5,Athletic,17
6,Foodie,1
7,Shopper,21


#### Student

In [33]:
user_categories[user_categories['Student'] == 1].sum().reset_index()

,index,0
0,Woman,684
1,Parent,29
2,Pet Owner,332
3,Gamer,18
4,Student,693
5,Athletic,331
6,Foodie,40
7,Shopper,677


#### Athletic

In [34]:
athletic = user_categories[user_categories['Athletic'] == 1]
athletic.sum().reset_index()

,index,0
0,Woman,338
1,Parent,19
2,Pet Owner,171
3,Gamer,17
4,Student,331
5,Athletic,346
6,Foodie,12
7,Shopper,330


#### Foodie

In [35]:
foodie = user_categories[user_categories['Foodie'] == 1]
foodie.sum().reset_index()

,index,0
0,Woman,40
1,Parent,2
2,Pet Owner,20
3,Gamer,1
4,Student,40
5,Athletic,12
6,Foodie,41
7,Shopper,38


#### Shopper

In [36]:
shopper = user_categories[user_categories['Shopper'] == 1]
shopper.sum().reset_index()

,index,0
0,Woman,679
1,Parent,35
2,Pet Owner,332
3,Gamer,21
4,Student,677
5,Athletic,330
6,Foodie,38
7,Shopper,689
